In [ ]:
# Get the data from the server. Run this once
!curl http://grfia.dlsi.ua.es/homus/HOMUS.zip > HOMUS.zip

In [ ]:
# Use this cell for shell commands, if necessary

In [92]:
import os
import pandas as pd
import sys
from skimage import data, io, filters

In [54]:
def process_file(fileobj, exid):
    X = None
    y = None
    
    # The first line of the file is the note-type
    notetype = fileobj.readline()
    y = notetype.rstrip("\n")
#     print(y)

    # The rest of the lines are semi-colon separated point tuples, e.g. 5,6;1,5;8;9 etc.
    xstring = ""
    
    for line in fileobj:
        xstring += line.rstrip()
#     print(xstring)

    # Split the string based on the semicolon
    points_partition = xstring.strip(";").split(";")
    
    # Split the string based on the commas
    # The result is a doubly nested list, e.g. [['5','6'], ['1', '5'], ['8', '9'] ... ] etc
    points_split = [item.split(",") for item in points_partition]
    
    # Convert strings to ints. TODO: Make this a list comprehension instead of a for loop.
    for tup in points_split:
        tup[0] = int(tup[0])
        tup[1] = int(tup[1])


    # Create a DataFrame from the result
    df = pd.DataFrame(points_split, columns = ["x1", "x2"])
    df['id'] = exid
    df['y'] = y
    return df

In [78]:
INDIR = "./HOMUS"

CAPCOUNT = float("inf") # <---- Change this to a set number for testing; to do everything set this high
capflag = False

# Each file will return a DataFrame; store these all in a list
allcoordslist = [] 

# this will be used as a progress indicator
count = 1;

for root, dirs, filenames in os.walk(INDIR):
    print("Working on " + str(count))
    count += 1
    
    for counter, f in enumerate(filenames):
        if counter > CAPCOUNT:
            capflag = True
            break
        else:
            fileobj = open(os.path.join(root, f), 'r')
            readresults = process_file(fileobj, f[:-4]) # Provide the filename (w/o '.txt') as a unique identifier
            fileobj.close()
            allcoordslist.append(readresults)
    if capflag:
        break

# Concatenate the list of DataFrames into one large dataframe. This is much, much, much, much more efficient
#     than doing this is the list body.
allcoords = pd.concat(allcoordslist)

Working on 1
Working on 2
Working on 3
Working on 4
Working on 5
Working on 6
Working on 7
Working on 8
Working on 9
Working on 10
Working on 11
Working on 12
Working on 13
Working on 14
Working on 15
Working on 16
Working on 17
Working on 18
Working on 19
Working on 20
Working on 21
Working on 22
Working on 23
Working on 24
Working on 25
Working on 26
Working on 27
Working on 28
Working on 29
Working on 30
Working on 31
Working on 32
Working on 33
Working on 34
Working on 35
Working on 36
Working on 37
Working on 38
Working on 39
Working on 40
Working on 41
Working on 42
Working on 43
Working on 44
Working on 45
Working on 46
Working on 47
Working on 48
Working on 49
Working on 50
Working on 51
Working on 52
Working on 53
Working on 54
Working on 55
Working on 56
Working on 57
Working on 58
Working on 59
Working on 60
Working on 61
Working on 62
Working on 63
Working on 64
Working on 65
Working on 66
Working on 67
Working on 68
Working on 69
Working on 70
Working on 71
Working on 72
W

In [81]:
# Save the DataFrame to a CSV, just in case something terrible happens.
allcoords.to_csv("homus_all.csv")

In [105]:
def round_to(x, nearest = 50):
    return int(nearest * round(float(x)/nearest))

# Get the dimension of the image based on the max of the tuples. Note that some files have negative coordinates, so 
x1_dim = round_to(max(allcoords['x1']) + abs(min(allcoords['x1'])), 25)
x2_dim = round_to(max(allcoords['x2']) + abs(min(allcoords['x2'])), 25)
                  
allcoords['x1f'] = allcoords['x1'] + abs(min(allcoords['x1'])) 
allcoords['x2f'] = allcoords['x2'] + abs(min(allcoords['x2']))

In [112]:
allcoords.to_csv("homus_all.csv")

In [94]:
def gen_pixel_array(df_in):
    

array([[ 47, 123, 133, ...,  14,   3,  12],
       [ 93, 144, 145, ...,  12,   7,   7],
       [126, 147, 143, ...,   2,  13,   3],
       ..., 
       [ 81,  79,  74, ...,   6,   4,   7],
       [ 88,  82,  74, ...,   5,   7,   8],
       [ 91,  79,  68, ...,   4,  10,   7]], dtype=uint8)